# Prédiction du nombre de nouveaux étudiants en L1

## 1. Prétraitement des données

- Importation des librairies
- Importation du jeu de données
- Nettoyage des données:
    1. Verification s'il y a des données manquantes au jeu de données

In [82]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn

Récupéraion du dataset

In [83]:
df = pd.read_csv("datas/prop_admi_post_bac.csv",sep=";")

Visualisation des données

In [84]:
df.head(5)

,Année du Baccalauréat,Enseignements de spécialité,Formation,Nombre de candidats bacheliers ayant confirmé au moins un vœu,Nombre de candidats bacheliers ayant reçu au moins une proposition d'admission,Nombre de candidats bacheliers ayant accepté une proposition d'admission
0,2021,"Art,Art","Licence Electronique, Génie électrique, EEA",0,0.0,0
1,2021,"Art,Art",Licence Formation générale aux métiers de l in...,0,0.0,0
2,2021,"Art,Art",Licence LEA,0,0.0,0
3,2021,"Art,Art",Licence Mathématiques,0,0.0,0
4,2021,"Art,Art",Licence Mathématiques et informatique,0,0.0,0


Nous vérifions si une colonne comporte des données manquantes. Comme nous pouvons le voir, aucune ligne n'est vide.

In [85]:
df.isna().sum()

Année du Baccalauréat                                                             0
Enseignements de spécialité                                                       0
Formation                                                                         0
Nombre de candidats bacheliers ayant confirmé au moins un vœu                     0
Nombre de candidats bacheliers ayant reçu au moins une proposition d'admission    0
Nombre de candidats bacheliers ayant accepté une proposition d'admission          0
dtype: int64

Renommage des colonnes par soucis de simplification lors du traitement.

In [86]:
df.rename(columns={"Année du Baccalauréat" : "annee",
           "Enseignements de spécialité" : "spe",
           "Formation": "formation",
           "Nombre de candidats bacheliers ayant confirmé au moins un vœu" : "voeu_confirme",
           "Nombre de candidats bacheliers ayant reçu au moins une proposition d'admission" : "prop_recu",
           "Nombre de candidats bacheliers ayant accepté une proposition d'admission" : "prop_accepte"}, inplace=True)
df.head(5)

,annee,spe,formation,voeu_confirme,prop_recu,prop_accepte
0,2021,"Art,Art","Licence Electronique, Génie électrique, EEA",0,0.0,0
1,2021,"Art,Art",Licence Formation générale aux métiers de l in...,0,0.0,0
2,2021,"Art,Art",Licence LEA,0,0.0,0
3,2021,"Art,Art",Licence Mathématiques,0,0.0,0
4,2021,"Art,Art",Licence Mathématiques et informatique,0,0.0,0


Vérification et correction au besoin des types de données.

Ici on remarque que prop_recu est de type float64. Aucune des valeurs de la colonne n'est de type float, on peu donc passer la colonne en Int.

In [87]:
df.dtypes

annee              int64
spe               object
formation         object
voeu_confirme      int64
prop_recu        float64
prop_accepte       int64
dtype: object

In [88]:
df["prop_recu"].value_counts().unique()

array([4022, 1091,  641,  412,  325,  257,  228,  216,  169,  141,  132,
        117,  115,   91,   87,   82,   78,   75,   74,   71,   65,   60,
         59,   52,   48,   46,   43,   42,   40,   39,   37,   36,   35,
         34,   29,   27,   25,   23,   22,   21,   20,   19,   18,   17,
         16,   15,   14,   13,   12,   11,   10,    9,    8,    7,    6,
          5,    4,    3,    2,    1])

In [89]:
df["prop_recu"] = df["prop_recu"].astype(int)

In [90]:
df.dtypes

annee             int64
spe              object
formation        object
voeu_confirme     int64
prop_recu         int64
prop_accepte      int64
dtype: object

On sait que le nouveau bac demande à choisir une spécialité dominante et une spécialité secondaire. On remarque que la colonne spe sépare ses données par une virgule, mais parfois le nom d'une spécialité contient une virgule. On remarque également que lorsque la virgule sépare deux spécialités (Art,Art), il n'y a pas d'espace après celle-ci, et que quand la virgule est dans le nom d'une spécialité, (Histoire-Géographie, Géopolitique et Sciences politiques), il y a un espace après la virgule. 

Cette information pourra être importante si nous décidons de séparer la colonne des spécialités et de regarder en fonction d'une spécialité en particulier plutôt qu'une combinaison de spécialité. Pour le moment il n'est pas utile de traiter plus que cela cette colonne, mais c'est à garder dans un coin pour plus tard au besoin.

In [91]:
df["spe"].unique()

array(['Art,Art',
       'Art,Histoire-Géographie, Géopolitique et Sciences politiques',
       'Art,Humanités, Littérature et Philosophie',
       'Art,Langues, littératures et cultures étrangères et régionales',
       'Art,Mathématiques Spécialité',
       'Art,Numérique et Sciences Informatiques',
       'Art,Physique-Chimie Spécialité',
       'Art,Sciences Economiques et Sociales Spécialité',
       "Art,Sciences de l'ingénieur et sciences physiques",
       'Art,Sciences de la vie et de la Terre Spécialité',
       'Biologie/Ecologie,Histoire-Géographie, Géopolitique et Sciences politiques',
       'Biologie/Ecologie,Humanités, Littérature et Philosophie',
       'Biologie/Ecologie,Langues, littératures et cultures étrangères et régionales',
       'Biologie/Ecologie,Mathématiques Spécialité',
       'Biologie/Ecologie,Physique-Chimie Spécialité',
       'Biologie/Ecologie,Sciences Economiques et Sociales Spécialité',
       'Histoire-Géographie, Géopolitique et Sciences politiq

## Lecture des données

Nous avons 6 variables différentes dans notre dataset :

- annee :
    - L'année du passage du BAC pour les étudiants
    - type *int*, [2021-2023]
- spé :
    - Spécialités lors du passage du BAC
    - type *(string, string)*: (spé1,spé2)
- formation : 
    - Formation postulés par les étudiants, 
    - type *string*
- voeu_accepté :
    - Nombre de voeux confirmés par les étudiants
    - type *int*, [0, inf]
- prop_recu :
    - Nombre de propositions d'admission reçus par les étudiants
    - type *int*, [0, prop_reçu]
- prop_accepte :
     - Nombre de proposition d'admission acceptés par les étudiants
    - type *int*, [0, voeu_accepté]